In [1]:
# Quick hack to load local library code
import os

os.chdir(os.path.join(os.getcwd(), ".."))

# Load API key and secret from environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd
import statsmodels.api as sm

from sklearn.metrics import accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
import validmind as vm

# local clb168jzz0000928h07nbs1o9
# staging clb5bf15i000b1smvawuoi4u0

vm.init(
    api_host="https://api.staging.vm.validmind.ai/api/v1/tracking",
    project="clb5bf15i000b1smvawuoi4u0"
    # project="clb168jzz0000928h07nbs1o9"
)

True

In [4]:
df = pd.read_csv("notebooks/insurance_mortality/insurance_dataset.csv")

targets = vm.DatasetTargets(
    target_column="Number_Of_Deaths",
    description="Number of deaths for a group of policyholders",
)

analyze_results = vm.analyze_dataset(
    dataset=df,
    dataset_type="training",
    targets=targets,
)

Analyzing dataset...
Pandas dataset detected.
Inferring dataset types...
Preparing in-memory dataset copy...
Calculating field statistics...
Calculating feature correlations...
Generating correlation plots...
Successfully logged dataset metadata and statistics.
Running data quality tests...
Running data quality tests for "training" dataset...

Preparing dataset for tests...
Preparing in-memory dataset copy...


  0%|          | 0/6 [00:00<?, ?it/s]

Skipping class_imbalance test because no target column is defined


100%|██████████| 6/6 [00:02<00:00,  2.68it/s]



Test suite has completed.
Sending results to ValidMind...
Successfully logged test results for test: duplicates
Successfully logged test results for test: cardinality
Successfully logged test results for test: missing
Successfully logged test results for test: skewness
Successfully logged test results for test: zeros

Summary of results:

Test         Passed      # Passed    # Errors    % Passed
-----------  --------  ----------  ----------  ----------
duplicates   False              0           1           0
cardinality  True               6           0         100
missing      True               9           0         100
skewness     False              2           1     66.6667
zeros        False              0           1           0



In [5]:
model = sm.load('notebooks/insurance_mortality/mortality_model.pickle')

In [6]:
train_df = pd.read_csv("notebooks/insurance_mortality/train_df.csv")
test_df = pd.read_csv("notebooks/insurance_mortality/test_df.csv")

In [7]:
x_train = train_df.drop("Number_Of_Deaths", axis=1)
y_train = train_df.loc[:, "Number_Of_Deaths"].astype(int)

x_test = test_df.drop("Number_Of_Deaths", axis=1)
y_test = test_df.loc[:, "Number_Of_Deaths"].astype(int)

In [8]:
# This fails at "Computing model evaluation metrics..." when using a statsmodels model
eval_results = vm.evaluate_model(
    model,
    train_set=(x_train, y_train),    
    val_set=(x_test, y_test),    
    test_set=(x_test, y_test),
)

Logging model metadata and parameters...
Extracting training/validation set metrics from trained model...
Refitting model...
Successfully logged metrics
Running model evaluation tests...
Generating model predictions on test dataset...
Generating model predictions on training dataset...
Computing model evaluation metrics...


  0%|          | 0/6 [00:00<?, ?it/s]


AttributeError: 'GLMResults' object has no attribute 'coef_'